This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

FSXS7-5f-6N-JTfUF16g


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import requests
import json

response = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/')

print(response)

<Response [200]>


Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

data = response.json()
data.keys()

dict_keys(['dataset'])

In [4]:
dataset = data['dataset']
dataset.keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [5]:
dataset['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [6]:
dataset['data'][0]

['2020-12-01',
 112.2,
 112.2,
 111.5,
 112.0,
 None,
 51.0,
 5703.0,
 None,
 None,
 None]

In [7]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# The JSON structured data is a dictionary with one key 'dataset' and its corresponding value, which is another dictionary.
# The inner dictionary consists of 11 sets of keys and values.
# For my further analysis, I only require the values associated with the 'column_names' and 'data' keys
# as well as their respective values.

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [8]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
# (keep in mind that the date format is YYYY-MM-DD).
# 2. Convert the returned JSON object into a Python dictionary.
data_2017 = []
dict_date = {}

for n in dataset['data']:
    if '2017' in n[0]:
        for a, b in zip(dataset['column_names'], n):
            dict_date[a]= b
        data_2017.append(dict_date)
        dict_date = {}
        
data_2017[0]

{'Date': '2017-12-29',
 'Open': 51.76,
 'High': 51.94,
 'Low': 51.45,
 'Close': 51.76,
 'Change': None,
 'Traded Volume': 34640.0,
 'Turnover': 1792304.0,
 'Last Price of the Day': None,
 'Daily Traded Units': None,
 'Daily Turnover': None}

In [9]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

open = {}
for n in data_2017:
    if isinstance(n['Open'], float):
        open[n['Date']] = n['Open']
        
open_hignest = max(open, key = open.get)
open_lowest = min(open, key = open.get)

print('The highest opening prices is $', open[open_hignest], 'in', open_hignest)
print('The lowest opening prices is $', open[open_lowest], 'in', open_lowest)

The highest opening prices is $ 53.11 in 2017-12-14
The lowest opening prices is $ 34.0 in 2017-01-24


In [10]:
# 4. What was the largest change in any one day (based on High and Low price)?

change = {}
for n in data_2017:
    if isinstance(n['High'], float) and isinstance(n['Low'], float):
        change[n['Date']] = n['High'] - n['Low']

change_max = max(change, key = change.get)
print('The largest change in any one day is', change[change_max], 'in', change_max)

The largest change in any one day is 2.8100000000000023 in 2017-05-11


In [11]:
# 5. What was the largest change between any two days (based on Closing Price)?

close = []
for n in data_2017:
    close.append(n['Close'])

close_max = 0
for i in range(len(close)-1):
    if isinstance(close[i], float) and isinstance(close[i+1], float):
        diff = abs(close[i] - close[i+1])
        if diff > close_max:
            close_max = diff
print('The largest change between any two days is', diff)

The largest change between any two days is 0.3200000000000003


In [12]:
# 6. What was the average daily trading volume during this year?

daily = []
for n in data_2017:
    daily.append(n['Traded Volume'])
    
daily_ave = sum(daily)/len(daily)
print('The average daily trading volume during 2017 is' , daily_ave)

The average daily trading volume during 2017 is 89124.33725490196


In [13]:
# 7. What was the median trading volume during this year. 
# (Note: you may need to implement your own function for calculating the median.)

daily_sort = sorted(daily)
len(daily)/2

127.5

In [14]:
daily_median = daily_sort[128]
print('The median trading volume during 2017 is' , daily_median)

The median trading volume during 2017 is 76600.0
